In [1]:
import json
import os
import openai
import random
import copy


# client = openai.OpenAI()

In [2]:
current_question = {
    "section_name": "self-check-questions",
    "section_text": "Question: \n2. \n\n  What are the drawbacks to analyzing the global economy on a regional basis?\n  \n Answer: 2. \n\n  A region can have some of high-income countries and some of the low-income countries. Aggregating per capita real GDP will vary widely across countries within a region, so aggregating data for a region has little meaning. For example, if you were to compare per capital real GDP for the United States, Canada, Haiti, and Honduras, it looks much different than if you looked at the same data for North America as a whole. Thus, regional comparisons are broad-based and may not adequately capture an individual country’s economic attributes.\n  \n",
    "chapter_name": "32-self-check-questions",
    "textbook_name": "principles-economics-3e",
    "chapter_url": "https://openstax.org/books/principles-economics-3e/pages/32-self-check-questions",
    "question": "\n2. \n\n  What are the drawbacks to analyzing the global economy on a regional basis?\n  \n",
    "answer": "2. \n\n  A region can have some of high-income countries and some of the low-income countries. Aggregating per capita real GDP will vary widely across countries within a region, so aggregating data for a region has little meaning. For example, if you were to compare per capital real GDP for the United States, Canada, Haiti, and Honduras, it looks much different than if you looked at the same data for North America as a whole. Thus, regional comparisons are broad-based and may not adequately capture an individual country’s economic attributes.\n  \n"
}

In [59]:
prompt_extract_features = f"""
Question: {current_question['question']}
Gold Answer: {current_question['answer']}
Your task: Think about the question and understand the answer first, then extract complete bullet points that are keyfeatures based on the answer for rebuilding the answer from scratch for the same question, more features will be helpful.
Return a JSON object in this exact format:
        {{
            "Analysis": "Analysis of the question, answer, and the features",
            "Features": [
                {{"feature": "string", "detail": "string"}},
                {{"feature": "string", "detail": "string"}},
                ... and so on, until all features are covered.
            ]
        }}
"""
temperature = 0.0

response_extract_features = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                # {"role": "system", "content": "You are an economics export and a knowledgeable teacher. Think concisely and precisely. If you meet difficulty, ask for clarification. pause and think deeply before you answer."},
                {"role": "user", "content": prompt_extract_features}
            ],
            temperature=temperature,
            # max_tokens=2000,
            response_format={"type": "json_object"}
        )

gpt_answer_extract_features = response_extract_features.choices[0].message.content
json_extract_features = json.loads(gpt_answer_extract_features)


In [ ]:
# random choice 3 feature
random_features = random.sample(json_extract_features['Features'], k=3)
print(random_features)


In [ ]:
random_features

In [6]:
prompt_swap_feature_p1 = f"""
Task: For the selected feature, what is a misleading direction of the feature for the question. \
Modify or rewrite the feature to create a misleading feature. Keep other features the same.
Question: {current_question['question']}
"""


prompt_swap_feature_p3 = f"""
Return a JSON object in this exact format:
        {{
            "analysis_1": "How the misleading feature can be created. Think step by step.",
            "analysis_2": "Think deeper.",
            "new_feature": "string",
            "new_feature_detail": "string",
            "old_feature": "string",
            ]
        }}
"""

In [ ]:
swapped_feature_list = []

count = 0
for feature in random_features:
    print(f'{count}th feature: {feature}')
    count += 1
    prompt_swap_feature_p2 = ''
    prompt_swap_feature_p2 += f"Selected Feature: {feature['feature']}. Description: {feature['detail']}\n"
    # construct the prompt with the selected feature and the other unselected features
    for other_feature in json_extract_features['Features']:
        if feature['feature'] != other_feature['feature']:
            prompt_swap_feature_p2 += f"Other Feature: {other_feature['feature']}. Description: {other_feature['detail']}\n"

    prompt_swap_feature = prompt_swap_feature_p1 + prompt_swap_feature_p2 + prompt_swap_feature_p3

    response_swap_feature = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt_swap_feature}
        ],
        temperature=temperature,
        response_format={"type": "json_object"}
    )
    gpt_answer_swap_feature = response_swap_feature.choices[0].message.content
    swapped_feature_list.append(gpt_answer_swap_feature)


In [ ]:
temp = json.loads(swapped_feature_list[0])
temp


In [ ]:
# for each swapped feature, find the corresponding feature in the original features, and swap the feature and detail
new_features_list  = []

for swapped_feature in swapped_feature_list:
    new_feature = json.loads(swapped_feature)
    # copy the original features
    new_features = copy.deepcopy(json_extract_features['Features'])
    for feature in new_features:
        if feature['feature'] == new_feature['old_feature']:
            feature['feature'] = new_feature['new_feature']
            feature['detail'] = new_feature['new_feature_detail']
    new_features_list.append(new_features)

new_features_list

In [ ]:
# for each new_features, construct the prompt of the question with the new features

prompt_generate_choice_p1 = f"""
Features:
"""

prompt_generate_choice_p2 = f"""
Task: Generate the answer for the given question with the given features, all of them are high level features and you need follow the directions
Question: {current_question['question']}
You do not need absolutely follow the directions, use the directions as a guide. aggregate the all the separate answers into a comprehensive answer in one paragraph. Do not need to do summary in the end.
"""

answer_list = []

# generate the answer with the original features
prompt_generate_choice = ''
for temp in json_extract_features['Features']:
    temp_str = f"{temp['feature']}: {temp['detail']} \n"
    prompt_generate_choice += temp_str
prompt_generate_choice = prompt_generate_choice_p1 + prompt_generate_choice
prompt_generate_choice = prompt_generate_choice + prompt_generate_choice_p2

response_generate_choice = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt_generate_choice}],
    temperature=temperature,
)

gpt_answer_generate_choice = response_generate_choice.choices[0].message.content
answer_list.append(gpt_answer_generate_choice)

# generate the answer with the new features
for new_features in new_features_list:
    prompt_generate_choice = ''
    for temp in new_features:
        temp_str = f"{temp['feature']}: {temp['detail']} \n"
        prompt_generate_choice += temp_str
    prompt_generate_choice = prompt_generate_choice_p1 + prompt_generate_choice
    prompt_generate_choice = prompt_generate_choice + prompt_generate_choice_p2
    #print(prompt_generate_choice)
    
    # generate the answer
    response_generate_choice = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_generate_choice}],
        temperature=temperature,
    )

    gpt_answer_generate_choice = response_generate_choice.choices[0].message.content
    print('==='*20)
    print(gpt_answer_generate_choice)
    answer_list.append(gpt_answer_generate_choice)


In [34]:

import random

def shuffle_and_track_first_index(lst):
    # Store the first element
    first_element = lst[0]
    
    # Shuffle the list
    random.shuffle(lst)
    
    # Find the new index of the original first element
    new_index = lst.index(first_element)
    
    return lst, new_index



In [ ]:
new_suffle_answer_list, new_index = shuffle_and_track_first_index(answer_list)

print(new_index)

In [ ]:
print(current_question['question'])
for i, c in enumerate('ABCD'):
    print(f'{c}: {new_suffle_answer_list[i]}\n')


